# 1. Data Preparation
Before starting to train the model, some steps need to be done.

# 1.1. ROI Extraction
The images are large in size (above 2000 x 2000 pixels), so it is better to extract the area that is important to us by using a cropping function. This cropping process should be applied to both the image and the respective masks.

<div style="text-align: center;">
    <img src="Images/image-11.png" alt="Alt text" style="display: block; margin: 0 auto;">
</div>


* TASK:

In [ ]:
### Code
###Cropping to ROI
###Import folder contains images for Roughness Analysis 
import numpy as np
import cv2
from matplotlib import pyplot as plt 
import statistics
import os 
from PIL import Image

folder = r'D:\02-Project\01-Wear Detection\10_Training_Dataset\Masks\raw_masks_converted' 

###Necessary path 
Main_path = r'C:\Users\karimi' 

###Exporting images and names and lists 
names = [] 
images = [] 
c = 0 

###Find all images 
for filename in os.listdir(folder): 
            img = cv2.imread(os.path.join(folder,filename)) 
            if img is not None: 
                    images.append(img) 
                    c+=1 
                    names.append(filename) 

print('number of images: ' + str(c)) 

os.chdir(folder) 
    
for i in range(len(images)): 
    print(names[i]) 
    imgTest = cv2.imread(names[i])  
    imgheight=imgTest.shape[0]
    imgwidth=imgTest.shape[1]
    cropped_image = imgTest[int(imgwidth/2)-1000:int(imgwidth/2)+1000, int(imgheight/2)-700:int(imgheight/2)+700] # Slicing to crop the image
    cropped = '%s_new_cropped.png'%names[i][:-4] 
    cv2.imwrite(cropped, cropped_image) 

     

os.chdir(Main_path)

In [ ]:
import cv2
import os

# Input and output directories
int_folder = r'D:\08-Studi-Veranstaltungen\KIP\Jupyter-Notebook\TestKIP'
out_folder = r'D:\08-Studi-Veranstaltungen\KIP\Jupyter-Notebook\TestKIP\out'

for filename in os.listdir(int_folder): 
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        # Open the image
        img_path = os.path.join(int_folder, filename)
        imgTest = cv2.imread(img_path)  
        imgheight = imgTest.shape[0]
        imgwidth = imgTest.shape[1]
        cropped_image = imgTest[int(imgwidth/2)-1000:int(imgwidth/2)+1000, int(imgheight/2)-700:int(imgheight/2)+700] # Slicing to crop the image

        # Output filename with "_cropped" added
        output_filename = os.path.splitext(filename)[0] + "_cropped.png"

        # Save the cropped image to the output directory
        output_path = os.path.join(out_folder, output_filename)
        cv2.imwrite(output_path, cropped_image)

# 1.2. Image Format Converting 
It is recommended to convert the format of both the image and mask to TIFF format, which is suitable for the recommended Convolutional Neural Network (CNN) model.

In [ ]:
### Code
##ConTif
input_folder = r'D:\02-Project\01-Wear Detection\10_Training_Dataset\Masks\raw_masks_converted_cropped'
output_folder = r'D:\02-Project\01-Wear Detection\10_Training_Dataset\Masks\raw_masks_converted_cropped_tif'

for filename in os.listdir(input_folder):
    if filename.endswith(('.jpg', '.jpeg', '.png')):  # Add other image formats if needed
        # Open the image
        img_path = os.path.join(input_folder, filename)
        im = Image.open(img_path)

        # Resize the image to (512, 512)
        #im_resized = im.resize((512, 512))

        # Save the resized image in TIFF format to the output folder
        output_path = os.path.join(output_folder, os.path.splitext(filename)[0] + '.tif')
        im.save(output_path, 'TIFF')

print("Conversion completed.")


# 1.3. Image Augmentation

After extracting the ROI and converting the image and mask to TIFF format, we need to increase our dataset size using augmentation techniques. Here's how you can implement this method, ensuring that the image name and the respective mask name are the same:

* Define Augmentation Parameters: Determine the augmentation techniques to apply, such as rotation, flipping, scaling, etc.

* Loop Through Images: Iterate through each image and its corresponding mask.

* Apply Augmentation: Apply the defined augmentation techniques to both the image and its mask.

* Save Augmented Images: Save the augmented images and their masks with the same names as the original images and masks.

<div style="text-align: center;">
    <img src="Images/image-12.png" alt="Alt text" style="display: block; margin: 0 auto;">
</div>

In [ ]:
### Code
import numpy as np
from matplotlib import pyplot as plt
from skimage.transform import AffineTransform, warp
from skimage import io, img_as_ubyte
import random
import os
from scipy.ndimage import rotate

#Define functions for each operation
def rotation(image, seed):
    random.seed(seed)
    angle= 180
    r_img = rotate(image, angle, mode='reflect', reshape=False, order=0)
    return r_img

def h_flip(image, seed):
    hflipped_img= np.fliplr(image)
    return  hflipped_img

def v_flip(image, seed):
    vflipped_img= np.flipud(image)
    return vflipped_img

def v_transl(image, seed):
    random.seed(seed)
    n_pixels = random.randint(-64,64)
    vtranslated_img = np.roll(image, n_pixels, axis=0)
    return vtranslated_img

def h_transl(image, seed):
    random.seed(seed)
    n_pixels = random.randint(-64,64)
    htranslated_img = np.roll(image, n_pixels, axis=1)
    return htranslated_img

transformations = {'rt': rotation,
                      'hf': h_flip, 
                      'vf': v_flip
                 }                #use dictionary to store names of functions

images_path= r"D:\02-Project\01-Wear Detection\10_Training_Dataset\Images\raw_images_cropped_tif/" #path to original images
masks_path = r"D:\02-Project\01-Wear Detection\10_Training_Dataset\Masks\raw_masks_converted_cropped_tif/"
img_augmented_path= r"D:\02-Project\01-Wear Detection\10_Training_Dataset\Images\raw_images_cropped_tif_aug/" # path to store aumented images
msk_augmented_path= r"D:\02-Project\01-Wear Detection\10_Training_Dataset\Masks\raw_masks_converted_cropped_tif_aug/" # path to store aumented images
images=[] # to store paths of images from folder
images_name=[]
masks=[]
masks_name=[]

for im in os.listdir(images_path):  # read image name from folder and append its path into "images" array     
    images.append(os.path.join(images_path,im))
    images_name.append(im) 

for msk in os.listdir(masks_path):  # read image name from folder and append its path into "images" array     
    masks.append(os.path.join(masks_path,msk))
    masks_name.append(msk) 


